**Задание 2. Тестирование нейронной сети.**

Во второй части второй тетрадки показано, как обучить нейронную сеть предсказывать морфологические тэги. Однако предсказывается только часть речи и одушевленность. Ваша задача добавить другие категории (лицо, падеж, время и т.д.), а также оценить общее качество полученного тэггера.
(ЗАМЕЧАНИЕ: если вы будете добавлять остальные категории так, как это сделано в тетрадке для одушевленности - добавлением отдельного классифицирующего выхода (головы) - у вас получится, что выходы независимы, т.е. предсказать прош. время и вин. падеж - вполне возможно, поэтому не забудьте добавить класс "пустой категории", а также сделать простую эвристику на предсказаниях (if tags['POS'] == 'VERB': tags['Case'] = ''. Если вы найдете другой способ решить эту проблему, не забудьте написать об этом в отчете) 

Критерии оценки:
1) добавлены оставшиеся грамматические категории, модель обучена (следите, чтобы лосс снижался, а метрики росли, останавливайте обучение когда лосс и метрики на трейне и валидации начинают сильно расходиться) - 4 балла
2) на отложенной выборке (или кросс-валидацией) посчитана метрика, учитывающая сразу все категории (например, как жаккар в первой тетрадке) - 4 балла
3) есть исправление несовместимости некоторых категории (эвристикой или как-то более глобально)  - 2 балла

# 0. Все загружаем

In [0]:
from lxml import etree
from collections import defaultdict

In [2]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.zip

--2019-05-24 11:09:41--  http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.zip
Resolving opencorpora.org (opencorpora.org)... 148.251.2.141
Connecting to opencorpora.org (opencorpora.org)|148.251.2.141|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2190887 (2.1M) [application/zip]
Saving to: ‘annot.opcorpora.no_ambig_strict.xml.zip.1’

annot.opcorpora.no_ 100%[===================>]   2.09M  --.-KB/s    in 0.1s    

2019-05-24 11:09:41 (20.4 MB/s) - ‘annot.opcorpora.no_ambig_strict.xml.zip.1’ saved [2190887/2190887]



In [3]:
!unzip annot.opcorpora.no_ambig_strict.xml.zip

Archive:  annot.opcorpora.no_ambig_strict.xml.zip
replace annot.opcorpora.no_ambig_strict.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())
corpus = open('corpus_train.txt', 'w')
vocab = defaultdict(set)
tags = set()

for sentence in open_corpora.xpath('//tokens'):
    length = len(sentence.xpath('token'))
    ended = False
    for i,token in enumerate(sentence.xpath('token')):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        
        if (i+1)==length and gram_info[0] == 'PNCT':
            gram_info = ['SENT']
            ended = True
        
            
        corpus.write(word[0] + '\t' + ','.join(gram_info) + '\n')
        lemma = token.xpath('tfr/v/l/@t')[0]
        vocab[word[0].lower()].add((','.join(gram_info), lemma.lower()))
        tags.add(','.join(gram_info))
    
    if not ended:
        corpus.write('.\tSENT\n')
f = open('lexicon.txt', 'w')

for word in vocab:
    f.write(word + '\t')
    f.write('\t'.join([' '.join(pair) for pair in vocab[word]]))
    f.write('\n')
# f.write('SENT\tSENT .')
f.close()
f = open('open_class.txt', 'w')

f.write('\n'.join([tag for tag in tags if 'NOUN' in tag or 'VERB' in tag or 'ADJF' in tag]))
f.close()

# 1. Делаем модель

In [5]:
import keras
from collections import Counter

Using TensorFlow backend.


In [0]:
corpus = []
vocab = defaultdict(set)
tags = set()

for sentence in open_corpora.xpath('//tokens'):
    sent = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        
        
            
        sent.append(word + gram_info)
    corpus.append(sent)

In [7]:
corpus[0]

[['«', 'PNCT'],
 ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'],
 ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'],
 ['»', 'PNCT'],
 ['учит', 'VERB', 'impf', 'tran', 'sing', '3per', 'pres', 'indc'],
 ['прикусить', 'INFN', 'perf', 'tran'],
 ['язык', 'NOUN', 'inan', 'masc', 'sing', 'accs']]

In [8]:
#Отложим часть для будущей метрики. 
metric_corpus = corpus[int(len(corpus)*0.97):]
corpus = corpus[:int(len(corpus)*0.97)]
len(corpus), len(metric_corpus)

(10272, 318)

**Далее добавляются следующие категории:**: лицо, падеж, время, число, переходность.



## Создаем словари для признаков

In [9]:
vocab = Counter()
poses = Counter()

for sent in corpus:
  for word, pos, *tags in sent:
    vocab[word.lower()] += 1
    poses[pos] += 1
vocab = {word for word,c in vocab.most_common() if c > 3}

len(vocab), len(poses)

(2122, 22)

In [0]:
#Просто слова
id2word = {i+2:word for i, word in enumerate(vocab)}
id2word[0] = '<PAD>'
id2word[1] = '<UNK>'
word2id = {word:i for i, word in id2word.items()}

In [0]:
#Часть речи
id2pos = {i+1:pos for i, pos in enumerate(poses)}
id2pos[0] = '<PAD>'
pos2id = {pos:i for i, pos in id2pos.items()}

In [0]:
#Одушевленность
id2anim = {1:'NONE', 2:'anim', 3:'inan'}
id2anim[0] = '<PAD>'
anim2id = {tag:i for i, tag in id2anim.items()}

In [0]:
#Ссылка на тэги: http://opencorpora.org/dict.php?act=gram
#Лицо
id2face = {1:'NONE', 2:'masc', 3:'femn', 4:'neut', 5:'ms-f'}
id2face[0] = '<PAD>'
face2id = {tag:i for i, tag in id2face.items()}

In [0]:
#Падеж
id2case = {1:'NONE', 2:'nomn', 3:'gent', 4:'datv', 
           5:'accs', 6:'ablt', 7:'loct'}
id2case[0] = '<PAD>'
case2id = {tag:i for i, tag in id2case.items()}

In [0]:
#Число
id2num = {1:'NONE', 2:'sing', 3:'plur'}
id2num[0] = '<PAD>'
num2id = {tag:i for i, tag in id2num.items()}

In [0]:
#Переходность
id2tran = {1:'NONE', 2:'tran', 3:'intr'}
id2tran[0] = '<PAD>'
tran2id = {tag:i for i, tag in id2tran.items()}

In [0]:
sents_ids = []
poses_ids = []
anim_ids = []
face_ids = []
case_ids = []
num_ids = []
tran_ids = []

for sent in corpus:
    sents_ids.append([word2id.get(word.lower(), 1) for word, *_ in sent])
    poses_ids.append([pos2id[tag] for word, tag, *_ in sent])
    
    anim_ids_sent = []
    face_ids_sent = []
    case_ids_sent = []
    num_ids_sent = []
    tran_ids_sent = []   
    
    
    for word, pos, *tags in sent:
      anim_tag, face_tag, case_tag, num_tag, tran_tag = 1, 1, 1, 1, 1
      for tag in tags:
        
        if tag in anim2id:
          anim_tag = anim2id[tag]
        if tag in face2id:
          face_tag = face2id[tag]
        if tag in case2id:
          case_tag = case2id[tag]
        if tag in num2id:
          num_tag = num2id[tag]
        if tag in tran2id:
          tran_tag = tran2id[tag]
      
      if pos == 'VERB':
        anim_tag = 1
        case_tag = 1
      elif pos == "NOUN":
        tran_tag = 1
      elif pos == "PNCT" or pos == "ADV":
        anim_tag, face_tage, case_tag, num_tag, tran_tag = 1, 1, 1, 1, 1
      elif pos == "ADJF":
        tran_tag, anim_tag == 1
      
      anim_ids_sent.append(anim_tag)
      face_ids_sent.append(face_tag)
      case_ids_sent.append(case_tag)
      num_ids_sent.append(num_tag)
      tran_ids_sent.append(tran_tag)
    anim_ids.append(anim_ids_sent)
    face_ids.append(face_ids_sent)
    case_ids.append(case_ids_sent)
    num_ids.append(num_ids_sent)
    tran_ids.append(tran_ids_sent)

## Подготавливаем модель

In [0]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, LSTM, Input, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

In [0]:
MAX_LENGTH = 10
sents_ids_padded = pad_sequences(sents_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
poses_ids_padded = pad_sequences(poses_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
anim_ids_padded = pad_sequences(anim_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
face_ids_padded = pad_sequences(face_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
case_ids_padded = pad_sequences(case_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
num_ids_padded = pad_sequences(num_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')
tran_ids_padded = pad_sequences(tran_ids, maxlen=MAX_LENGTH, padding='post',  truncating='post')

In [0]:
train_index, test_index = train_test_split(list(range(sents_ids_padded.shape[0])),test_size=0.2)

In [0]:
train_sents, test_sents = sents_ids_padded[train_index], sents_ids_padded[test_index]
train_pos, test_pos = poses_ids_padded[train_index], poses_ids_padded[test_index]
train_anim, test_anim = anim_ids_padded[train_index], anim_ids_padded[test_index]
train_face, test_face = face_ids_padded[train_index], face_ids_padded[test_index]
train_case, test_case = case_ids_padded[train_index], case_ids_padded[test_index]
train_num, test_num = num_ids_padded[train_index], num_ids_padded[test_index]
train_tran, test_tran = tran_ids_padded[train_index], tran_ids_padded[test_index]

In [22]:
inp = Input(shape=(MAX_LENGTH, ))
# создаем эбмеддинги размерностью 8
x = Embedding(len(word2id), 8)(inp)

# пропускаем эмбединги через LSTM, чтобы модель учитывала контекст
x = LSTM(20, return_sequences=True)(x)

# каждый выход - последовательность классов
pos = TimeDistributed(Dense(len(pos2id), activation='softmax'))(x)
anim = TimeDistributed(Dense(len(anim2id), activation='softmax'))(x)
face = TimeDistributed(Dense(len(face2id), activation='softmax'))(x)
case = TimeDistributed(Dense(len(case2id), activation='softmax'))(x)
num = TimeDistributed(Dense(len(num2id), activation='softmax'))(x)
tran = TimeDistributed(Dense(len(tran2id), activation='softmax'))(x)

 
model = Model(inputs=inp, outputs=[pos, anim, face, case, num, tran], 
           )
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from keras.utils import to_categorical

In [24]:
model.fit(x=train_sents, 
        y=[to_categorical(train_pos), to_categorical(train_anim), to_categorical(train_face), to_categorical(train_case),
          to_categorical(train_num), to_categorical(train_tran)], 
          batch_size=128, epochs=150, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 6573 samples, validate on 1644 samples
Epoch 1/150
6573/6573 [==============================] - 3s 496us/step - loss: 10.7121 - time_distributed_1_loss: 3.0527 - time_distributed_2_loss: 1.3280 - time_distributed_3_loss: 1.7407 - time_distributed_4_loss: 1.9815 - time_distributed_5_loss: 1.3407 - time_distributed_6_loss: 1.2685 - time_distributed_1_acc: 0.3638 - time_distributed_2_acc: 0.4287 - time_distributed_3_acc: 0.2907 - time_distributed_4_acc: 0.3704 - time_distributed_5_acc: 0.4134 - time_distributed_6_acc: 0.4307 - val_loss: 9.5721 - val_time_distributed_1_loss: 2.8192 - val_time_distributed_2_loss: 1.1923 - val_time_distributed_3_loss: 1.6042 - val_time_distributed_4_loss: 1.7169 - val_time_distributed_5_loss: 1.2084 - val_time_distributed_6_loss: 1.0311 - val_time_distributed_1_acc: 0.4835 - val_time_distributed_2_acc: 0.4333 - val_time_distributed_3_acc: 0.3262 - val_time_distributed_4_acc: 0.4310 - val_time_distribut

# 2. Тестируем результаты

In [25]:
#Просто пример работы модели
test_samples = [
    "эти типы стали есть на складе .".split()
]

test_samples_X = []
for sent in test_samples:
    sent_ids = []
    for word in sent:
        if word not in word2id:
            #print(word)
            pass
        sent_ids.append(word2id.get(word.lower(), 1))
    test_samples_X.append(sent_ids)
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
predictions = model.predict(test_samples_X)
print(type(predictions))
print(type(predictions[0]))
for i, idx in enumerate(predictions[0].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2pos[idx])
print('')

for i, idx in enumerate(predictions[1].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2anim[idx])
print('')

for i, idx in enumerate(predictions[2].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2face[idx])
print('')

for i, idx in enumerate(predictions[3].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2case[idx])
  
print('')
for i, idx in enumerate(predictions[4].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2num[idx])
  
print('')
for i, idx in enumerate(predictions[5].argmax(axis=2)[0]):
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2tran[idx])

<class 'list'>
<class 'numpy.ndarray'>
эти ADJF
типы NOUN
стали VERB
есть ADJF
на PREP
складе NOUN
. PNCT

эти NONE
типы inan
стали NONE
есть NONE
на NONE
складе inan
. NONE

эти NONE
типы masc
стали NONE
есть NONE
на NONE
складе femn
. NONE

эти nomn
типы nomn
стали NONE
есть NONE
на NONE
складе loct
. NONE

эти plur
типы plur
стали plur
есть plur
на NONE
складе sing
. NONE

эти NONE
типы NONE
стали intr
есть NONE
на NONE
складе NONE
. NONE


In [26]:
for i, idx in enumerate(predictions[0].argmax(axis=2)[0]):
  #print(predictions[0].argmax(axis=2)[0]) #Части речи предложения
  #print(predictions[1].argmax(axis=2)[0]) #Одушевленность
  #print(predictions[2].argmax(axis=2)[0]) #Лицо
  #print(predictions[3].argmax(axis=2)[0]) #Падеж
  #print(predictions[4].argmax(axis=2)[0]) #Число
  #print(predictions[5].argmax(axis=2)[0]) #Переходность
  if i > (len(test_samples[0])-1):
    break
  print(test_samples[0][i], id2pos[idx])
print('')

эти ADJF
типы NOUN
стали VERB
есть ADJF
на PREP
складе NOUN
. PNCT



In [27]:
#Считаем Метрику между множествами тэгов исходных и полученных. 
corpus[0]


[['«', 'PNCT'],
 ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'],
 ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'],
 ['»', 'PNCT'],
 ['учит', 'VERB', 'impf', 'tran', 'sing', '3per', 'pres', 'indc'],
 ['прикусить', 'INFN', 'perf', 'tran'],
 ['язык', 'NOUN', 'inan', 'masc', 'sing', 'accs']]

Дорабатываем качество, чтобы не было лишних тэгов.

In [0]:
#VERB: no anim, no case
#NOUN: no tran
#PNCT: no anim, no case, no tran, no face, no num
#ADJF: no tran, no anim
#ADV: no tran, no face, no case, no anim, no num

In [0]:
#На выходе словарь, где ключ - порядковый номер токена в предложении, а значения - список его тэгов. 
#Все NONE удалены, т.к. для подсчета метрики они будут очень занижать все, ведь в исходных тэгах нет NONE
def ready_for_predict(sent):
  d = {} #Наш словарь
  test_sample = []
  test_sample.extend([word[0] for word in sent])
  test_samples=[test_sample]
  #print(test_samples)
  test_samples_X = []
  for sent in test_samples:
    sent_ids=[]
    for word in sent:
      if word not in word2id:
        pass
      sent_ids.append(word2id.get(word.lower(), 1))
    test_samples_X.append(sent_ids)
  test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
  predictions = model.predict(test_samples_X)
  
  
  #Для каждой категории добавляем значение в словарь d
  for i, idx in enumerate(predictions[0].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    d[i] = []
    d[i].append(id2pos[idx])
    #print(test_samples[0][i], id2pos[idx])
  
  for i, idx in enumerate(predictions[1].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    
    if d[i][0] == 'VERB' or d[i][0] == 'PNCT' or d[i][0]=='ADVB' or d[i][0] == 'ADJF' or d[i][0] == "PREP":
      d[i].append('NONE')
    else:
      d[i].append(id2anim[idx])  
    #print(test_samples[0][i], id2anim[idx])
    #print('')

  for i, idx in enumerate(predictions[2].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break    
    if d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP":
      pass
    else:  
      d[i].append(id2face[idx])
    #print(test_samples[0][i], id2face[idx])
    #print('')

  for i, idx in enumerate(predictions[3].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break    
    if d[i][0] == 'VERB' or d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP":
      pass
    else:  
      d[i].append(id2case[idx])
    #print(test_samples[0][i], id2case[idx])
    #print('')
  
  for i, idx in enumerate(predictions[4].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    if d[i][0] == 'PNCT' or d[i][0] == 'ADVB' or d[i][0] == "PREP":
      pass
    else:
      d[i].append(id2num[idx])
    #print(test_samples[0][i], id2num[idx])  
    #print('')
  
  for i, idx in enumerate(predictions[5].argmax(axis=2)[0]):
    if i > (len(test_samples[0])-1):
      break
    if d[i][0] != 'VERB':
      pass
    else:
      d[i].append(id2tran[idx])
    #print(test_samples[0][i], id2tran[idx])
    
    for key in d.keys():      
      while 'NONE' in d[key]:
        d[key].remove('NONE')    
    
  return d   

Далее мы считаем метрику Жаккара для каждого слова в предложении, затем из этого выбираем среднее и добавляем в список metrics. 

In [30]:
import numpy as np

metrics = []
count_full = 0
mistakes = []

for sent in metric_corpus:

  sent_metrics = []
  #print(sent)
  sent_real = []
  words = []
  if len(sent) > 10:
    sent = sent[:10]
  for word in sent:
    sent_real.append(word[1:])
    #print(word[0])
    words.append(word[0])
  #print(words)
  sent_pred = ready_for_predict(sent)
  
  
  for i in range(0, len(sent_real)):
    set_real = set(sent_real[i])     
    set_pred = set(sent_pred[i])
    metric = len(set_real&set_pred) / len(set_pred|set_real)
    sent_metrics.append(metric)
  
  if np.mean(sent_metrics) == 1:
    count_full += 1
  if np.mean(sent_metrics) < 0.5:
    mistakes.append(words)
  
  if len(sent_metrics)==0:
    #print("EMPTYTYTYTYTYTYTYT")
    continue
  metrics.append(np.mean(sent_metrics))
  
#print(sent_real)
#print(sent_pred.keys())
print(len(metrics))
print(len(metric_corpus))

318
318


In [31]:
print('Метрика Жаккара: ', np.mean(metrics))

Метрика Жаккара:  0.7330689666041755


In [33]:
count_full

17

In [34]:
mistakes

[['Занимаешься', 'паломничествами', '?'],
 ['Он',
  'лечит',
  'болезнями',
  'или',
  'какими-то',
  'жизненными',
  'испытаниями',
  '.'],
 ['«', 'Esta', 'noche', 'es', 'Nochebuena', ',', 'Y', 'no', 'es', 'noche'],
 ['Свободную', 'демократическую', 'Украину', '.'],
 ['Сумчатая', 'куница'],
 ['Она', 'давеча', 'собиралась', '.'],
 ['Философия', 'потока'],
 ['Растущая', 'Индонезия'],
 ['Растущая', 'Индонезия'],
 ['Страдают', 'экспортёры', '.'],
 ['Уехал', 'Димка', '.'],
 ['Сломалась', 'стиральная', 'машинка', '.'],
 ['Освободители'],
 ['Первопроходцами', 'станут', 'уральцы', '.'],
 ['iCamp', 'предлагает', 'массу', 'возможностей', ':'],
 ['Стивен', 'Хокинг', 'выдвинул', 'новую', 'научную', 'теорию'],
 ['Economist', 'продан'],
 ['ICE', 'купил', 'NYSE']]

In [35]:
for mistake in mistakes:
  print(ready_for_predict(mistake))
  

{0: ['NOUN', 'nomn', 'sing'], 1: ['NOUN', 'femn', 'sing'], 2: ['PNCT']}
{0: ['PREP'], 1: ['NOUN', 'inan', 'femn', 'loct', 'sing'], 2: ['NOUN', 'inan', 'femn', 'loct', 'sing'], 3: ['CONJ'], 4: ['PREP'], 5: ['NOUN', 'inan', 'masc', 'datv', 'sing'], 6: ['CONJ'], 7: ['PNCT']}
{0: ['PNCT'], 1: ['LATN'], 2: ['LATN'], 3: ['LATN'], 4: ['LATN'], 5: ['PNCT'], 6: ['LATN'], 7: ['LATN'], 8: ['LATN'], 9: ['LATN']}
{0: ['PREP'], 1: ['NOUN', 'inan', 'femn', 'ablt', 'sing'], 2: ['PREP'], 3: ['PNCT']}
{0: ['PREP'], 1: ['PREP']}
{0: ['PREP'], 1: ['NOUN', 'inan', 'femn', 'loct', 'sing'], 2: ['PREP'], 3: ['PNCT']}
{0: ['NOUN', 'nomn', 'sing'], 1: ['NOUN', 'femn', 'sing']}
{0: ['NOUN', 'nomn', 'sing'], 1: ['CONJ']}
{0: ['NOUN', 'nomn', 'sing'], 1: ['CONJ']}
{0: ['PREP'], 1: ['NOUN', 'inan', 'femn', 'ablt', 'sing'], 2: ['PNCT']}
{0: ['PREP'], 1: ['NOUN', 'inan', 'masc', 'gent', 'plur'], 2: ['PNCT']}
{0: ['PREP'], 1: ['PREP'], 2: ['NOUN', 'inan', 'masc', 'gent', 'plur'], 3: ['PNCT']}
{0: ['PREP']}
{0: ['NOUN'